<a href="https://www.kaggle.com/code/lousacco/sacco-gen-ai-intensive-course-capstone-2025q1?scriptVersionId=234714975" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Google Gen AI Intensive Course Capstone 2025Q1

## Overview

Employers and insurance providers spend significant time manually entering detailed benefits data into HRIS systems when setting up open enrollment. This data, usually captured in Summary of Benefits and Coverage (SBC) PDF documents, must be painstakingly re-entered, creating tedious and error-prone work. Additionally, employees often find it challenging to choose the most suitable medical, dental, or vision plan because comparing detailed benefits across multiple documents can be confusing. As a result, decisions frequently default to cost rather than overall value.

Generative AI (GenAI) can address these pain points effectively by utilizing Retrieval-Augmented Generation (RAG). With a RAG-based system, SBC documents are ingested and embedded into a vector store at plan creation time. Employees can then interact through a user-friendly chat interface during open enrollment, asking specific questions about each plan. The system can dynamically generate comparison tables or personalized recommendations based on individual needs. This project illustrates how leveraging GenAI capabilities learned in this course can significantly simplify and improve benefits plan selection.

## Goal
This notebook demonstrates a Retrieval-Augmented Generation (RAG) pipeline leveraging Generative AI (Google Gemini) to extract structured, meaningful data from uploaded SBC PDFs, enabling easy and transparent comparison of insurance plans. Furthermore, this can serve as an example for any industry that relies heavily on PDF documents to share information with its clients. The objective is to illustrate what's possible with this use case that is easily extended to others.

### Gen AI Capabilities Utilized

-   **Embeddings & Vector Store:** Utilized `GoogleGenerativeAIEmbeddings` to create vector representations of SBC document text chunks, storing them in a `Chroma` vector database for efficient retrieval.
-   **Vector Search/Vector Store/Vector Database:** Employed `Chroma` as a vector database and performed similarity searches (`store.similarity_search`) to find document chunks relevant to specific queries (including a multi-query approach).
-   **Retrieval Augmented Generation (RAG) & Grounding:** Implemented a multi-query RAG pipeline that retrieves relevant document chunks (context) from the vector store and provides this context to an LLM, instructing it to base its answers *only* on this retrieved information, ensuring grounded responses.
-   **Document Understanding:** Processed PDF documents (SBCs) using `PyMuPDF`, extracted text content, and used the LLM to interpret this text to extract specific data points based on context.
-   **Structured Output (JSON mode/controlled generation):** Engineered detailed prompts, including schema definitions and strict formatting rules, to guide the LLM in generating responses formatted as valid JSON objects containing extracted SBC details.
-   **Few-Shot Prompting:** Included a concrete example of the desired JSON output structure within the LLM prompt to improve the accuracy and formatting of the generated response based on the provided context.

Let's get started and for any issues don't hesitate to contact me directly through my Kaggle [profile](https://www.kaggle.com/lousacco).

## Set-up SDK and Packages

The first step is to set-up the SDK and include the required packages I'll use throughout this notebook. Note that there may be some dependency conflicts, perhaps due to some already installed packages that come with Kaggle. These are innocuous and do not affect the running of the rest of the project.

In [1]:
# Remove unused conflicting packages
!pip uninstall -qqy jupyterlab kfp 2>/dev/null

# Upgrade pip first (Good Practice)
!pip install -q -U pip

# Install/Upgrade all directly required packages together
# The -U flag will upgrade dependencies if needed by these packages
!pip install -q -U \
    google-genai \
    langchain-community \
    PyMuPDF \
    chromadb \
    langchain-google-genai \
    langchain-chroma \
    tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 91.5 MB/s eta 0:00:00
ERROR: p

## Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [2]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

print("🔑 GOOGLE_API_KEY loaded:", "OK" if GOOGLE_API_KEY else "MISSING")

🔑 GOOGLE_API_KEY loaded: OK


If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

## Embedding Model Selection Criteria

For this project involving embedding PDF chunks (Summary of Benefits and Coverage documents) to build a Retrieval Augmented Generation (RAG) system for structured data extraction, the `models/text-embedding-004` model was selected over other available options (`models/embedding-001`, experimental models) for the following key reasons:

1.  **Enhanced Retrieval Performance:** As a newer generation model, `text-embedding-004` generally demonstrates superior performance on retrieval benchmarks (like MTEB) compared to the older `models/embedding-001`. Better retrieval accuracy is crucial for RAG, as it ensures more relevant context is provided to the language model, leading to more accurate and complete answers.
2.  **Stability and General Availability:** While experimental models (`models/gemini-embedding-exp-*`) might offer potentially higher performance based on recent research, they lack the stability guarantees of a generally available (GA) model. `text-embedding-004` is a stable GA release, making it a more reliable choice for consistent development and potential future use compared to experimental versions that may change or have limitations.
3.  **Suitability for RAG Task:** This model is explicitly designed for semantic understanding and tasks like information retrieval and semantic search, which are fundamental to RAG. It also supports optional `task_type` parameters (e.g., `retrieval_document`, `retrieval_query`) that can further optimize embeddings specifically for the document chunking and querying stages of the RAG workflow.

In conclusion, `models/text-embedding-004` provides a compelling combination of improved performance over older stable models and greater reliability than experimental versions, making it the most suitable choice for embedding the SBC documents in this RAG application.

In [3]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

for model in client.models.list():
  if 'embedContent' in model.supported_actions:
    print(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


## Verify Simple Embedding & ChromaDB Write

Before ingesting all PDFs, we run a quick “hello world” embedding into a temporary ChromaDB (`./chroma_test_db`) to confirm:

1. **Embedding & API connectivity** – that `GoogleGenerativeAIEmbeddings` instantiates correctly and your key works.  
2. **Filesystem & Chroma write** – that we have write permissions in `/kaggle/working/` and Chroma can persist data.

Why this matters:

- **Fail‑Fast**: Catch core problems (readonly FS, bad key) in seconds, not minutes into a full ingestion.  
- **Environment Sanity**: Kaggle VMs occasionally have stale mounts or quota glitches—this verifies the session is healthy.  
- **Cleanliness**: We isolate and then delete `./chroma_test_db`, so our real vector store stays pristine.

Once this passes, we confidently proceed to split, embed, and store all SBC PDFs into Chroma.


In [4]:
import os
import shutil

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

def sanity_check_embedding_and_chroma(
    api_key: str,
    model_name: str = "models/text-embedding-004",
    test_text: str = "hello world",
    persist_dir: str = "./chroma_test_db",
    collection_name: str = "test_collection"
):
    """
    Sanity‑check embeddings and ChromaDB write access:
      1. Embed a single string.
      2. Write & read from a temporary Chroma vector store.
      3. Clean up the temporary directory.
    """
    # 1) Verify API key & initialize embedder
    print("🔑 GOOGLE_API_KEY loaded:", "OK" if api_key else "MISSING")
    embedder = GoogleGenerativeAIEmbeddings(
        model=model_name,
        google_api_key=api_key,
        task_type="retrieval_query"
    )
    print("✅ Embedding client initialized.")

    # 2) Embed test string
    vector = embedder.embed_query(test_text)
    print(f"✅ Embed dims: {len(vector)}")

    # 3) Write to temporary Chroma
    os.makedirs(persist_dir, exist_ok=True)
    store = Chroma.from_texts(
        texts=[test_text],
        embedding=embedder,
        persist_directory=persist_dir,
        collection_name=collection_name
    )
    print(f"✅ Wrote single embedding to ChromaDB at '{persist_dir}', collection '{collection_name}'")

    # 4) Clean up
    try:
        shutil.rmtree(persist_dir)
        print(f"🗑️ Removed temporary ChromaDB directory: {persist_dir}")
    except Exception as e:
        print(f"⚠️ Could not remove '{persist_dir}': {e}")

if __name__ == "__main__":
    sanity_check_embedding_and_chroma(api_key=GOOGLE_API_KEY)


🔑 GOOGLE_API_KEY loaded: OK
✅ Embedding client initialized.
✅ Embed dims: 768
✅ Wrote single embedding to ChromaDB at './chroma_test_db', collection 'test_collection'
🗑️ Removed temporary ChromaDB directory: ./chroma_test_db


## RAG Pipeline Overview

A modular Retrieval‑Augmented Generation (RAG) workflow that transforms a collection of SBC PDFs into fully structured, per‑plan JSON outputs by combining vector search with controlled LLM prompting.

### 1. Document Ingestion & Indexing  
- **PDF Parsing & Chunking:** Load each policy PDF and split its text into overlapping segments (e.g. 500 characters with a 150‑character overlap) for manageable embedding and retrieval.  
- **Embedding into Vector Store:** Convert each text segment into a dense vector using Google’s text‑embedding‑004 model (with `task_type="retrieval_document"`) and save both the vector and its metadata (source file, plan identifier) in a Chroma database.  
- **Robust Writes:** Batch writes with retry logic and small pauses to ensure resilience against transient failures or rate limits.

### 2. Multi‑Query Context Retrieval  
- **Targeted Queries:** Define a collection of natural‑language questions—one per desired data field (e.g. effective dates, HSA availability, out‑of‑pocket limits, exclusions, links, etc.).  
- **Similarity Search:** For each question, retrieve the top K (e.g. 5) most relevant text chunks from Chroma.  
- **Context Consolidation:** Merge and deduplicate the results across all queries to build a single, comprehensive context pool that covers every aspect of each plan.

### 3. Per‑Plan JSON Assembly  
- **Grouping by Plan:** Use stored metadata to partition the retrieved context so that each plan’s chunks are handled independently.  
- **Prompt Engineering:** For each plan, concatenate its text segments and feed them into an LLM prompt that includes:  
  1. Instructions positioning the model as an expert extractor  
  2. A clear JSON schema definition  
  3. A concise few‑shot example  
  4. A strict “use only this context, output valid JSON” directive  
- **Structured Output & Parsing:** Invoke Gemini via `llm.invoke()`, strip any formatting fences, and parse the clean JSON into native objects—substituting `"N/A"` where fields are missing.

### Techniques & GenAI Capabilities

- **Embeddings & Vector Store**: Converts text chunks into vectors optimized for document retrieval; stored in Chroma (Capabilities: Embeddings; Vector search / vector store).  
- **Document Understanding**: Uses PyMuPDF to reliably extract and split PDF content (Capability: Document understanding).  
- **Multi‑Query RAG & Grounding**: Executes multiple targeted similarity searches and grounds the LLM’s output in retrieved context only (Capabilities: Retrieval-augmented generation; Grounding).  
- **Prompt Engineering for Structured Output**: Embeds a JSON schema and example in‑prompt to guarantee valid, machine‑parseable responses (Capability: Structured output / JSON mode).  
- **Few‑Shot Prompting**: Supplies an in‑prompt example to demonstrate desired format and content (Capability: Few‑shot prompting).  


In [5]:
import os
import glob
import fitz
import time
import shutil
from math import ceil

from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tenacity import retry, stop_after_attempt, wait_exponential

# ── 1) Config ─────────────────────────────────────────────────────────
PDF_DIR       = "/kaggle/input/sbc-documents-small-set/"
WORK_DIR      = "/kaggle/working/chroma_db"
EMBED_MODEL   = "models/text-embedding-004"
CHUNK_SIZE    = 500
CHUNK_OVERLAP = 150
BATCH_SIZE    = 20
BATCH_DELAY   = 0.2

# ── 2) Helpers ─────────────────────────────────────────────────────────
def extract_text(path):
    """Return full text of a PDF or empty on error."""
    try:
        doc = fitz.open(path)
        txt = "".join(p.get_text() for p in doc)
        doc.close()
        print(f"✔️ Extracted {len(txt)} chars from {os.path.basename(path)}")
        return txt
    except Exception as e:
        print(f"❌ Error reading {path}: {e}")
        return ""

def extract_plan_name_from_text(text, fallback):
    """Pull a 'Plan Name:' line or fallback to filename."""
    name = fallback
    try:
        for line in text.split("\n", 20):
            if line.strip().lower().startswith("plan name:"):
                candidate = line.split(":", 1)[1].strip()
                if candidate:
                    name = candidate
                    break
    except Exception as e:
        print(f"      ⚠️ Error extracting plan name: {e}")
    print(f"      ℹ️ Using plan identifier: '{name}'")
    return name

def main():
    # ── 3) Load secret & workspace prep ───────────────────────────────
    print("🔑 GOOGLE_API_KEY loaded:", "OK" if GOOGLE_API_KEY else "MISSING")
    
    # Re-create clean WORK_DIR
    if os.path.isdir(WORK_DIR):
        try:
            shutil.rmtree(WORK_DIR)
            print(f"🗑️ Deleted existing ChromaDB directory: {WORK_DIR}")
        except Exception as e:
            print(f"⚠️ Error deleting {WORK_DIR}: {e}")
    os.makedirs(WORK_DIR, exist_ok=True)

    # ── 4) Prepare splitter & embedder & store ────────────────────────
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found or empty. Cannot initialize embedder.")

    embedder = GoogleGenerativeAIEmbeddings(
        model=EMBED_MODEL,
        google_api_key=GOOGLE_API_KEY,
        task_type="retrieval_document"
    )
    store = Chroma(persist_directory=WORK_DIR, embedding_function=embedder)
    print(f"✅ Initialized Chroma store in: {WORK_DIR}")

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=5))
    def add_batch_with_retry(texts, metadatas):
        """Retryable wrapper around store.add_texts()."""
        store.add_texts(texts=texts, metadatas=metadatas)

    # ── 5) Gather PDFs ──────────────────────────────────────────────────
    pdfs = sorted(glob.glob(os.path.join(PDF_DIR, "*.pdf")))
    print(f"\n🎯 {len(pdfs)} PDFs found:")
    for p in pdfs:
        print("    •", os.path.basename(p))
    print()

    # ── 6) Process each PDF in batches ────────────────────────────────
    total_chunks_added = 0

    for pdf_path in pdfs:
        pdf_fn = os.path.basename(pdf_path)
        print(f"\n📄 Processing {pdf_fn}...")
        text = extract_text(pdf_path)
        if not text:
            print("   ⚠️ Skipping file due to extraction error or empty content.")
            continue

        plan_id = extract_plan_name_from_text(text, pdf_fn)
        base_meta = {
            "source_file": pdf_fn,
            "plan_identifier": plan_id
        }

        chunks = splitter.split_text(text)
        n_batch = ceil(len(chunks) / BATCH_SIZE)
        print(f"   Embedding {len(chunks)} chunks in {n_batch} batches...")

        for i in range(n_batch):
            start, end = i * BATCH_SIZE, (i + 1) * BATCH_SIZE
            batch_texts = chunks[start:end]
            batch_metas = [
                {**base_meta, "chunk_index_in_doc": start + j}
                for j in range(len(batch_texts))
            ]

            print(f"      ▶️ Batch {i+1}/{n_batch} ({len(batch_texts)} chunks)…", end=" ")
            try:
                add_batch_with_retry(batch_texts, batch_metas)
                total_chunks_added += len(batch_texts)
                print("OK")
            except Exception as e:
                print(f"FAILED! Error: {e}. Falling back to single-chunk mode...")
                for idx, chunk in enumerate(batch_texts):
                    single_meta = batch_metas[idx]
                    try:
                        store.add_texts(texts=[chunk], metadatas=[single_meta])
                        total_chunks_added += 1
                    except Exception as inner:
                        print(f"            ⚠️ chunk#{start + idx} error: {inner}")
            time.sleep(BATCH_DELAY)

        print(f"   ✅ Finished processing {pdf_fn} (auto-persisted).")

    # ── 7) Final Summary and Inspection ────────────────────────────────
    print(f"\n📊 Total chunks added across all files: {total_chunks_added}")

    print("\n🧐 Inspecting final directory contents:")
    for root, dirs, files in os.walk(WORK_DIR):
        indent = "  " * root.replace(WORK_DIR, "").count(os.sep)
        rel = os.path.relpath(root, WORK_DIR)
        print(f"{indent}{os.path.basename(root)}/")
        for f in files:
            print(f"{indent}  - {f}")

if __name__ == "__main__":
    main()

🔑 GOOGLE_API_KEY loaded: OK
✅ Initialized Chroma store in: /kaggle/working/chroma_db

🎯 3 PDFs found:
    • bluecross_anthem_hmo.pdf
    • bluecross_hsa.pdf
    • bluecross_ppo_250.pdf


📄 Processing bluecross_anthem_hmo.pdf...
✔️ Extracted 22045 chars from bluecross_anthem_hmo.pdf
      ℹ️ Using plan identifier: 'bluecross_anthem_hmo.pdf'
   Embedding 61 chunks in 4 batches...
      ▶️ Batch 1/4 (20 chunks)… OK
      ▶️ Batch 2/4 (20 chunks)… OK
      ▶️ Batch 3/4 (20 chunks)… OK
      ▶️ Batch 4/4 (1 chunks)… OK
   ✅ Finished processing bluecross_anthem_hmo.pdf (auto-persisted).

📄 Processing bluecross_hsa.pdf...
✔️ Extracted 23167 chars from bluecross_hsa.pdf
      ℹ️ Using plan identifier: 'bluecross_hsa.pdf'
   Embedding 65 chunks in 4 batches...
      ▶️ Batch 1/4 (20 chunks)… OK
      ▶️ Batch 2/4 (20 chunks)… OK
      ▶️ Batch 3/4 (20 chunks)… OK
      ▶️ Batch 4/4 (5 chunks)… OK
   ✅ Finished processing bluecross_hsa.pdf (auto-persisted).

📄 Processing bluecross_ppo_250.pdf...

In [6]:
import os
import json
import traceback
from collections import defaultdict

from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI
from langchain_chroma import Chroma

# ── Config ───────────────────────────────────────────────────────────────
WORK_DIR        = "/kaggle/working/chroma_db"
EMBED_MODEL     = "models/text-embedding-004"
LLM_MODEL_NAME  = "gemini-2.0-flash"
K_PER_QUERY     = 5
# Set these to be very deterministic to avoid hallucinations
TEMP, TOP_P, TOP_K = 0, 1.0, 1

QUERIES_FOR_RETRIEVAL = {
    "plan_id":       "What is the plan name, carrier name, plan code, and policy number?",
    "dates":         "What are the original effective date, start date, end date, and/or coverage date for the plan?",
    "location":      "What is the issuing state for this insurance plan?",
    "hsa_pcp":       "Does this plan offer an HSA (Health Savings Account)? Does it require PCP referrals?",
    "oop_max":       "What are the out‑of‑pocket maximums or limits for individual/family and in‑network/out‑of‑network services?",
    "oop_exclusions":"What costs, premiums, or services are excluded from the out‑of‑pocket limit?",
    "links":         "Are there any website links or URLs mentioned for benefits or providers?",
    "summary_info":  "Provide general details about benefits, deductibles, copays, and overall cost sharing."
}
QUERIES_FOR_RETRIEVAL["coverage_period"] = \
  "What is the coverage period of this plan? (e.g., 01/01/2024 - 12/31/2024)"
QUERIES_FOR_RETRIEVAL["dates"] = (
  "What is the coverage period (start and end dates) for the plan?"
)

JSON_SCHEMA = """{
  "carrierPlanName": "",
  "startDate": "",
  "endDate": "",
  "coveragePeriod": "",
  "issuingState": "",
  "summary": "",
  "links": [ { "label": "", "url": "" } ],
  "hsaOffered": null,
  "out_of_pocket_max_values": [
    { "limit_type": "Individual/Person (In‑Network)", "value": "" },
    { "limit_type": "Family (In‑Network)",             "value": "" },
    { "limit_type": "Individual/Person (Out‑of‑Network)", "value": "" },
    { "limit_type": "Family (Out‑of‑Network)",          "value": "" }
  ],
  "out_of_pocket_exclusions": ""
}"""

EXAMPLE_JSON = """{
  "carrierPlanName": "Example Gold HMO",
  "startDate": "2023-01-01",
  "endDate": "2023-12-31",
  "issuingState": "CA",
  "summary": "An example summary highlighting key benefits and cost structure, approx 250 chars long.",
  "links": [ { "label": "Summary of Benefits", "url": "http://example.com/sbc" } ],
  "hsaOffered": false,
  "out_of_pocket_max_values": [
    { "limit_type": "Individual/Person (In‑Network)", "value": "$4000" },
    { "limit_type": "Family (In‑Network)",             "value": "$8000" },
    { "limit_type": "Individual/Person (Out‑of‑Network)", "value": "$8000" },
    { "limit_type": "Family (Out‑of‑Network)",          "value": "$16000" }
  ],
  "out_of_pocket_exclusions": "Premiums, non‑covered services."
}"""

def load_api_key():
    key = GOOGLE_API_KEY
    if not key:
        print("⚠️ Warning: GOOGLE_API_KEY not found.")
    else:
        print("🔑 GOOGLE_API_KEY loaded.")
    return key

def init_embedder(api_key: str):
    try:
        emb = GoogleGenerativeAIEmbeddings(
            model=EMBED_MODEL,
            google_api_key=api_key,
            task_type="retrieval_query"
        )
        print("✅ Embedding client initialized.")
        return emb
    except Exception as e:
        print(f"❌ Failed to init embedder: {e}")
        traceback.print_exc()
        return None

def init_llm(
    api_key: str,
    temperature: float = 0.0,
    top_p: float = 1.0,
    top_k: int = 5
):
    """
    Initialize the LLM client with sampling params.
     - temperature: 0.0 = deterministic
     - top_p: nucleus sampling (0.0–1.0)
     - top_k: top‑k filtering
    """
    try:
        llm = GoogleGenerativeAI(
            model=LLM_MODEL_NAME,
            google_api_key=api_key,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        print(f"✅ LLM initialized (temp={temperature}, top_p={top_p}, top_k={top_k}).")
        return llm
    except Exception as e:
        print(f"❌ Failed to init LLM: {e}")
        traceback.print_exc()
        return None


def load_store(embedder):
    if not os.path.isdir(WORK_DIR) or not os.listdir(WORK_DIR):
        print(f"❌ Error: ChromaDB '{WORK_DIR}' missing or empty.")
        return None
    try:
        store = Chroma(persist_directory=WORK_DIR, embedding_function=embedder)
        count = store._collection.count()
        print(f"✅ ChromaDB loaded ({count} vectors).")
        return store if count > 0 else None
    except Exception as e:
        print(f"❌ Failed to load ChromaDB: {e}")
        traceback.print_exc()
        return None

def build_prompt(context: str) -> str:
    return (
        "You are an expert health insurance information extraction assistant.\n"
        "Extract ALL fields into the JSON object below using ONLY the provided context.\n\n"
        "JSON Schema:\n```json\n" + JSON_SCHEMA + "\n```\n\n"
        "Example:\n```json\n" + EXAMPLE_JSON + "\n```\n\n"
        "Context:\n---START---\n" + context + "\n---END---\n\n"
        "Output ONLY the JSON object.\n"
    )

def multi_query_retrieval(store):
    print(f"\n📄 Retrieving top {K_PER_QUERY} chunks per query ({len(QUERIES_FOR_RETRIEVAL)} queries)…")
    all_docs, seen = [], set()
    for key, q in QUERIES_FOR_RETRIEVAL.items():
        try:
            for doc in store.similarity_search(q, k=K_PER_QUERY):
                if doc.page_content not in seen:
                    seen.add(doc.page_content)
                    all_docs.append(doc)
        except Exception as e:
            print(f"❌ Retrieval error for '{key}': {e}")
            traceback.print_exc()
    print(f"  -> {len(all_docs)} unique chunks retrieved.")
    return all_docs

def group_by_plan(docs):
    grouped = defaultdict(list)
    for doc in docs:
        pid = doc.metadata.get("plan_identifier", "Unknown")
        grouped[pid].append(doc)
    print(f"\n📊 {len(grouped)} plans to process.")
    return grouped

def process_plans(store, llm):
    docs = multi_query_retrieval(store)
    if not docs:
        print("\n⚠️ No docs retrieved—skipping.")
        return

    plans = group_by_plan(docs)
    results = {}

    for pid, chunks in plans.items():
        print(f"\n--- Plan: {pid} ({len(chunks)} chunks) ---")
        context = "\n\n---\n\n".join(d.page_content for d in chunks)
        prompt = build_prompt(context)

        try:
            resp = llm.invoke(prompt)
            clean = resp.strip().removeprefix("```json").removesuffix("```").strip()
            data = json.loads(clean)

            # ── Replace null out‑of‑network values with "N/A" ────────────────
            for entry in data.get("out_of_pocket_max_values", []):
                if entry.get("value") is None:
                    entry["value"] = "N/A"
            
            print("\n✨ Parsed JSON:")
            print(json.dumps(data, indent=2))
            results[pid] = data
        except Exception as e:
            print(f"❌ Error for plan '{pid}': {e}")
            traceback.print_exc()
            results[pid] = {"error": str(e), "raw": resp}

    print("\n🏁 Done processing all plans.")
    return results


def main():
    print("--- RAG Query: Multi-Query Retrieval + Per-Plan JSON Output ---")
    api_key = load_api_key()
    if not api_key:
        return

    embedder = init_embedder(api_key)
    if not embedder:
        return

    store = load_store(embedder)
    if not store:
        return
    llm = init_llm(api_key, temperature=TEMP, top_p=TOP_P, top_k=TOP_K)
    if not llm:
        return

    process_plans(store, llm)

if __name__ == "__main__":
    main()

--- RAG Query: Multi-Query Retrieval + Per-Plan JSON Output ---
🔑 GOOGLE_API_KEY loaded.
✅ Embedding client initialized.
✅ ChromaDB loaded (192 vectors).
✅ LLM initialized (temp=0, top_p=1.0, top_k=1).

📄 Retrieving top 5 chunks per query (9 queries)…
  -> 21 unique chunks retrieved.

📊 3 plans to process.

--- Plan: bluecross_hsa.pdf (6 chunks) ---

✨ Parsed JSON:
{
  "carrierPlanName": "Anthem PPO HSA 3200/0",
  "startDate": "2024-07-01",
  "endDate": "2025-06-30",
  "coveragePeriod": "07/01/2024 - 06/30/2025",
  "issuingState": "CA",
  "summary": "Summary of Benefits and Coverage: What this Plan Covers & What You Pay for Covered Services. The SBC shows you how you and the plan would share the cost for covered health care services.",
  "links": [
    {
      "label": "network provider list",
      "url": "https://www.anthem.com/ca"
    }
  ],
  "hsaOffered": true,
  "out_of_pocket_max_values": [
    {
      "limit_type": "Individual/Person (In\u2011Network)",
      "value": "$3,425"


## Chatbot Simulation for Open Enrollment Plan Comparison

This section demonstrates how an employee might interact with a benefits chatbot during open enrollment: the system uses our RAG pipeline to extract structured JSON for each plan, then transforms that data into a clear, side‑by‑side comparison table highlighting out‑of‑pocket maxima and other key details so users don’t have to wade through multiple SBC PDFs.

**Driving the Comparison**  
We first invoke `compare_plans`, which under the hood runs a multi‑query retrieval over all schema‑targeted questions, groups the resulting text chunks by plan, and prompts the LLM to produce clean JSON for each plan. Progress is surfaced with a `tqdm` bar, and any missing out‑of‑network values are replaced with “N/A” to ensure completeness.

**Table Assembly & Styling**  
Next, we map each JSON field into human‑readable rows—plan name, HSA availability, in‑ and out‑of‑network out‑of‑pocket maxima, exclusions, and coverage dates—then build a Pandas DataFrame with plans as columns. We strip file extensions, convert booleans to “Yes”/“No,” normalize dates, and apply centered styling plus a bold caption to produce an employee‑friendly table right in the notebook.

**Chatbot Session Simulation**  
Finally, a simple `main()` function ties it all together: we initialize the embedder, store, and LLM with slightly relaxed sampling parameters (temperature, top_p, top_k), echo a simulated user request, print a friendly “Let me look into that for you…” response, and render the styled comparison table—just like a live chatbot session.


In [7]:
import os
import io
import json
import contextlib
from tqdm.notebook import tqdm
import pandas as pd
from IPython.display import display

def compare_plans(store, llm):
    """
    Runs RAG + LLM per plan (with tqdm), then returns
    a cleaned & styled DataFrame (no .pdf suffixes, Yes/No,
    normalized dates) with a caption title.
    """
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        docs  = multi_query_retrieval(store)
        plans = group_by_plan(docs)

    results = {}
    for pid, chunks in tqdm(plans.items(), desc="Processing plans", unit="plan"):
        ctx    = "\n\n---\n\n".join(d.page_content for d in chunks)
        prompt = build_prompt(ctx)
        resp   = llm.invoke(prompt)
        clean  = resp.strip().removeprefix("```json").removesuffix("```").strip()
        data   = json.loads(clean)
        for e in data.get("out_of_pocket_max_values", []):
            if e.get("value") is None:
                e["value"] = "N/A"
        results[pid] = data

    # Define rows
    sample      = next(iter(results.values()))
    limit_types = [v["limit_type"] for v in sample["out_of_pocket_max_values"]]

    row_defs = [
        ("Plan Name",   lambda d: d.get("carrierPlanName", "")),
        ("HSA Offered", lambda d: str(d.get("hsaOffered", ""))),
    ]
    for lt in limit_types:
        label = lt.replace("Individual/Person", "OOP Max (Indv)") \
                  .replace("Family", "OOP Max (Fam)")
        row_defs.append((
            label,
            lambda d, lt=lt: next(
                (x["value"] for x in d["out_of_pocket_max_values"] if x["limit_type"]==lt),
                ""
            )
        ))
    row_defs += [
        ("OOP Exclusions", lambda d: d.get("out_of_pocket_exclusions", "")),
        ("Issuing State",  lambda d: d.get("issuingState", "")),
        ("Start Date",     lambda d: d.get("startDate", "")),
        ("End Date",       lambda d: d.get("endDate", "")),
    ]

    # Build DataFrame
    plan_ids = list(results.keys())
    data     = {
        pid: [ extractor(results[pid]) for _, extractor in row_defs ]
        for pid in plan_ids
    }
    index = [ label for label, _ in row_defs ]
    df    = pd.DataFrame(data, index=index)

    # 1) Strip .pdf from headers
    df.columns = [ os.path.splitext(c)[0] for c in df.columns ]
    df.columns.name = "Plan"
    df.index.name   = ""

    # 2) Fallback Plan Name → filename (no .pdf)
    df.loc["Plan Name"] = [
        name if name else os.path.splitext(pid)[0]
        for name, pid in zip(df.loc["Plan Name"], plan_ids)
    ]

    # 3) True/False → Yes/No
    df.replace({"True": "Yes", "False": "No"}, inplace=True)

    # 4) Normalize dates to MM/DD/YYYY
    def fmt_date(x):
        try:
            dt = pd.to_datetime(x, errors="coerce")
            return dt.strftime("%m/%d/%Y") if not pd.isna(dt) else x
        except:
            return x

    df.loc["Start Date"] = df.loc["Start Date"].apply(fmt_date)
    df.loc["End Date"]   = df.loc["End Date"].apply(fmt_date)

    # 5) Style with caption
    styled = (
        df.style
          .set_caption("Out‑of‑Pocket Expensese & Plan Details Comparison")
          .set_properties(**{"text-align": "center"})
          .set_table_styles([
              {"selector": "th",      "props": [("text-align", "center")]},
              {"selector": "caption","props": [
                  ("font-weight",    "bold"),
                  ("font-size", "1.25rem"),
                  ("padding-bottom","0.75em")
              ]}
          ])
    )
    return styled

def main():
    # Set these to be a little more relaxed for better NLP, yet still deterministic
    TEMP, TOP_P, TOP_K = 0.2, 0.9, 8

    # Initialize
    api_key  = load_api_key()
    embedder = init_embedder(api_key)
    store    = load_store(embedder)
    llm      = init_llm(api_key, temperature=TEMP, top_p=TOP_P, top_k=TOP_K)
    
    # Echo the prompt
    print("\n🧑‍💼 User: Please compare all three plans and provide out‑of‑pocket‑expense summary in table format.\n")

    # Compare and display
    print(f"🤖 ChatBot: Let me look into that for you...")
    styled_df = compare_plans(store, llm)
    print()
    display(styled_df)

if __name__ == "__main__":
    main()


🔑 GOOGLE_API_KEY loaded.
✅ Embedding client initialized.
✅ ChromaDB loaded (192 vectors).
✅ LLM initialized (temp=0.2, top_p=0.9, top_k=8).

🧑‍💼 User: Please compare all three plans and provide out‑of‑pocket‑expense summary in table format.

🤖 ChatBot: Let me look into that for you...


Processing plans:   0%|          | 0/3 [00:00<?, ?plan/s]

Plan,bluecross_hsa,bluecross_ppo_250,bluecross_anthem_hmo
,,,
Plan Name,Anthem PPO HSA 3200/0,Anthem Classic PPO 250/20/10,Anthem Classic HMO 15/30/250 admit /275 OP/1FHW/07-24
HSA Offered,Yes,None,None
OOP Max (Indv) (In‑Network),"$3,425","$2,250","$2,000"
OOP Max (Fam) (In‑Network),"$6,850","$4,500","$4,000"
OOP Max (Indv) (Out‑of‑Network),"$7,000","$6,500",N/A
OOP Max (Fam) (Out‑of‑Network),"$14,000","$13,000",N/A
OOP Exclusions,"Premiums, balance-billing charges, and health care this plan doesn't cover.","Pre-Authorization Penalties, Premiums, balance-billing charges, and health care this plan doesn't cover.","Premiums, balance-billing charges, and health care this plan doesn't cover."
Issuing State,CA,CA,CA
Start Date,07/01/2024,07/01/2024,07/01/2024


## Stay in touch

As I continue to explore Google AIs capabilities for use cases at my own company, I encourage you to stay in touch.

You can follow me through my [substack](https://daisyhealthcare.substack.com/) and on [LinkedIn](https://www.linkedin.com/in/lsacco/).